In [1]:
import pandas as pd
from imblearn.combine import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from scipy.stats import randint as sp_randint
import numpy as np
from sklearn.grid_search import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


# 언더샘플링

* RandomUnderSampler

In [2]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : RandomUnderSampler')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = RandomUnderSampler(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


Sampling name : RandomUnderSampler
XGBoost :  {'n_estimators': 100}
train score: -0.878695627267162
test score: -1.2120638348033028
log loss: 1.2120638348033028

Wall time: 15.6 s


In [56]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = RandomUnderSampler(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 18.3 s


* TomekLinks

In [75]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : TomekLinks')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = TomekLinks(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


Sampling name : TomekLinks
XGBoost :  {'n_estimators': 100}
train score: -0.7251771088756608
test score: -0.8483367239716572
log loss: 0.8483367239716572

Wall time: 17.5 s


In [58]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = TomekLinks(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)
print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 20.3 s


* CondensedNearestNeighbour

In [6]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : CondensedNearestNeighbour')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = CondensedNearestNeighbour(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


Sampling name : CondensedNearestNeighbour
XGBoost :  {'n_estimators': 100}
train score: -0.9661839500334193
test score: -0.969635800686705
log loss: 0.969635800686705

Wall time: 39.5 s


In [59]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = CondensedNearestNeighbour(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 33.8 s


* OneSidedSelection

In [8]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : OneSidedSelection')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = OneSidedSelection(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)


Sampling name : OneSidedSelection
XGBoost :  {'n_estimators': 100}
train score: -0.7263688124250864
test score: -0.8492100672506717
log loss: 0.8492100672506717

Wall time: 17.1 s


In [60]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = OneSidedSelection(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_sel

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 20.3 s


* EditedNearestNeighbours

In [10]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : EditedNearestNeighbours')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = EditedNearestNeighbours(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


Sampling name : EditedNearestNeighbours
XGBoost :  {'n_estimators': 100}
train score: -0.2759586630901983
test score: -1.2348221376983555
log loss: 1.2348221376983555

Wall time: 16.9 s


In [68]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = EditedNearestNeighbours(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.
log loss: 0.8498913152328835

Wall time: 18.7 s


* NeighbourhoodCleaningRule

In [12]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : NeighbourhoodCleaningRule')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = NeighbourhoodCleaningRule(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


Sampling name : NeighbourhoodCleaningRule
XGBoost :  {'n_estimators': 100}
train score: -0.47400507761038224
test score: -1.0056581044604918
log loss: 1.0056581044604918

Wall time: 17.1 s


In [62]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = NeighbourhoodCleaningRule(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 19.8 s


# 오버샘플링

* RandomOverSampler

In [14]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : RandomOverSampler')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = RandomOverSampler(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


Sampling name : RandomOverSampler
XGBoost :  {'n_estimators': 100}
train score: -1.0228018723955208
test score: -1.0835411517442664
log loss: 1.0835411517442664

Wall time: 22.1 s


In [63]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = RandomOverSampler(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 21.5 s


* ADASYN

In [16]:
%%time

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : ADASYN')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = ADASYN(ratio = 'minority',random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)

Sampling name : ADASYN
XGBoost :  {'n_estimators': 100}
train score: -0.6644230309532677
test score: -0.8745810331058764
log loss: 0.8745810331058764

Wall time: 20.9 s


In [64]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y =  ADASYN(ratio = 'minority',random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 21.4 s


* SMOTE

In [18]:
%%time


def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : SMOTE')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = SMOTE(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)

Sampling name : SMOTE
XGBoost :  {'n_estimators': 100}
train score: -0.8277837696861409
test score: -0.9217500570947348
log loss: 0.9217500570947348

Wall time: 22.9 s


In [65]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = SMOTE(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8498913152328835

Wall time: 22 s


# 복합샘플링 

* SMOTEENN

In [20]:
%%time


def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : SMOTEENN')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = SMOTEENN(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)

Sampling name : SMOTEENN
XGBoost :  {'n_estimators': 100}
train score: -0.5231346654950687
test score: -1.366697772760151
log loss: 1.366697772760151

Wall time: 19.5 s


In [73]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle = True)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = SMOTEENN(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8503676246932516

Wall time: 20 s


* SMOTETomek

In [22]:
%%time


def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : SMOTETomek')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= { 'n_estimators' : [100]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = SMOTETomek(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)

Sampling name : SMOTETomek
XGBoost :  {'n_estimators': 100}
train score: -0.7944025310495435
test score: -0.9446675039917019
log loss: 0.9446675039917019

Wall time: 22.5 s


In [72]:
%%time
df_imb = pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle = True)

columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','YearMonth', 'Month' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.5, random_state=0, shuffle=True)
X, y = NeighbourhoodCleaningRule(random_state=0).fit_sample(X_train, y_train)
df_final = pd.DataFrame(X, columns=columns)

model_xgb_imb = xgboost.XGBClassifier(n_estimators=100)
model_xgb_imb.fit(df_final, y)

print('cv = 20 , split_test_size = 0.5')
print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb_imb, X_test, y_test, scoring="neg_log_loss", cv=cv))))
print()

cv = 20 , split_test_size = 0.5
log loss: 0.8505592053293443

Wall time: 19.9 s


In [23]:
%%time

df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y =  SMOTETomek(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

model_xgb = xgboost.XGBClassifier(n_estimators=100)
model_xgb.fit(final_X, y)

print('Sampling name : SMOTETomek')
pred_y = model_xgb.predict_proba(X_test)
print('train score: {}'.format(model_xgb.score(final_X, y)))
print('test score: {}'.format(model_xgb.score(X_test, y_test)))
print('log loss: {}'.format(log_loss(y_test, pred_y)))
print()

print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb, X_test, y_test, scoring="neg_log_loss", cv=KFold(20, shuffle=True)))))

Sampling name : SMOTETomek
train score: 0.7000801924619086
test score: 0.6130440204514278
log loss: 0.9446675039917019



ValueError: y_true and y_pred contain different number of classes 4, 5. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 2 3 4]